# CMDS 파일 연도별 분리 및 정렬

## 목적
- `createdAt` 타임스탬프 기준으로 연도별 파일 분리
- 각 연도 파일 내에서 `sourceId` 오름차순 정렬
- **줄 내용은 한 글자도 수정하지 않음**

In [1]:
import os
import re
from datetime import datetime
from collections import defaultdict
from datetime import datetime, timezone, timedelta

## 0. 경로 설정

In [2]:
# ========== 경로 설정 (필요시 수정) ==========
BASE_DIR = r'C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관'

CMDS_INPUT_PATH = os.path.join(BASE_DIR, 'documents_referrers_fixed.cmds')
OUTPUT_DIR = os.path.join(BASE_DIR, 'yearly')  # 연도별 파일 저장 폴더

# 출력 폴더 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"입력 파일: {CMDS_INPUT_PATH}")
print(f"출력 폴더: {OUTPUT_DIR}")

입력 파일: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\documents_referrers_fixed.cmds
출력 폴더: C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\새로 다 다시 이관\yearly


## 1. 파일 로드

In [3]:
with open(CMDS_INPUT_PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(f"총 줄 수: {len(lines)}개")

총 줄 수: 23320개


## 2. 추출 함수

In [4]:
def extract_created_at(line):
    """줄에서 createdAt 값 추출 (밀리초 타임스탬프)"""
    match = re.search(r'"createdAt":(\d+)', line)
    if match:
        return int(match.group(1))
    return None

def extract_source_id(line):
    """줄에서 sourceId 값 추출"""
    match = re.search(r'"sourceId":"([^"]+)"', line)
    if match:
        return match.group(1)
    return None

KST = timezone(timedelta(hours=9))

def timestamp_to_year(timestamp_ms):
    """밀리초 타임스탬프 → 연도 (한국시간)"""
    dt = datetime.fromtimestamp(timestamp_ms / 1000, tz=KST)
    return dt.year

# 테스트
test_line = lines[0]
print(f"createdAt: {extract_created_at(test_line)}")
print(f"sourceId: {extract_source_id(test_line)}")
print(f"연도: {timestamp_to_year(extract_created_at(test_line))}")

createdAt: 1609740886000
sourceId: doc_15167823_06
연도: 2021


## 3. 연도별 그룹화

In [5]:
# 연도별로 (sourceId, 원본줄) 저장
yearly_data = defaultdict(list)
parse_errors = []

for i, line in enumerate(lines):
    line = line.rstrip('\n')
    
    if not line.strip():
        continue
    
    created_at = extract_created_at(line)
    source_id = extract_source_id(line)
    
    if created_at is None:
        parse_errors.append((i+1, 'createdAt 없음', line[:100]))
        continue
    
    if source_id is None:
        parse_errors.append((i+1, 'sourceId 없음', line[:100]))
        continue
    
    year = timestamp_to_year(created_at)
    yearly_data[year].append((source_id, line))

print(f"연도별 문서 수:")
for year in sorted(yearly_data.keys()):
    print(f"  {year}년: {len(yearly_data[year])}개")

if parse_errors:
    print(f"\n파싱 에러: {len(parse_errors)}개")
    for line_no, err, preview in parse_errors[:5]:
        print(f"  Line {line_no}: {err}")

연도별 문서 수:
  2010년: 841개
  2011년: 855개
  2012년: 1280개
  2013년: 1582개
  2014년: 1504개
  2015년: 1367개
  2016년: 1528개
  2017년: 1963개
  2018년: 1661개
  2019년: 1478개
  2020년: 1075개
  2021년: 1331개
  2022년: 1818개
  2023년: 1442개
  2024년: 1811개
  2025년: 1784개


## 4. sourceId 기준 정렬

In [6]:
def sort_key(source_id):
    """
    sourceId 정렬 키
    예: doc_12345_06 → 숫자 부분으로 정렬
    """
    # 숫자 부분 추출 시도
    match = re.search(r'doc_(\d+)', source_id)
    if match:
        return int(match.group(1))
    # 숫자 없으면 문자열 그대로
    return source_id

# 각 연도별로 정렬
for year in yearly_data:
    yearly_data[year].sort(key=lambda x: sort_key(x[0]))

print("정렬 완료!")

# 정렬 확인 (첫 연도 샘플)
first_year = min(yearly_data.keys())
print(f"\n{first_year}년 첫 5개 sourceId:")
for source_id, _ in yearly_data[first_year][:5]:
    print(f"  {source_id}")

정렬 완료!

2010년 첫 5개 sourceId:
  doc_2002067_06
  doc_2002068_06
  doc_2002069_06
  doc_2002070_06
  doc_2002071_06


## 5. 검증

In [7]:
# 총 문서 수 확인
total_in_yearly = sum(len(v) for v in yearly_data.values())
total_original = len([l for l in lines if l.strip()])

print(f"원본 문서 수: {total_original}개")
print(f"분류된 문서 수: {total_in_yearly}개")
print(f"파싱 에러: {len(parse_errors)}개")
print(f"합계: {total_in_yearly + len(parse_errors)}개")

if total_original == total_in_yearly + len(parse_errors):
    print("\n✅ 문서 수 일치!")
else:
    print("\n❌ 문서 수 불일치!")

원본 문서 수: 23320개
분류된 문서 수: 23320개
파싱 에러: 0개
합계: 23320개

✅ 문서 수 일치!


In [8]:
# 원본 줄이 그대로인지 확인 (샘플)
print("원본 줄 무결성 확인 (샘플):")

# 원본에서 몇 개 랜덤 선택해서 yearly_data에 동일하게 있는지 확인
sample_indices = [0, 100, 500, 1000, 5000]
all_lines_in_yearly = []
for year in yearly_data:
    for source_id, line in yearly_data[year]:
        all_lines_in_yearly.append(line)

mismatch = 0
for idx in sample_indices:
    if idx < len(lines):
        orig = lines[idx].rstrip('\n')
        if orig.strip() and orig in all_lines_in_yearly:
            print(f"  Line {idx}: ✅ 일치")
        elif orig.strip():
            print(f"  Line {idx}: ❌ 불일치")
            mismatch += 1

if mismatch == 0:
    print("\n✅ 원본 줄 무결성 확인 완료!")

원본 줄 무결성 확인 (샘플):
  Line 0: ✅ 일치
  Line 100: ✅ 일치
  Line 500: ✅ 일치
  Line 1000: ✅ 일치
  Line 5000: ✅ 일치

✅ 원본 줄 무결성 확인 완료!


## 6. 연도별 파일 저장

In [9]:
saved_files = []

for year in sorted(yearly_data.keys()):
    filename = f"documents_{year}.cmds"
    filepath = os.path.join(OUTPUT_DIR, filename)
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for source_id, line in yearly_data[year]:
            f.write(line + '\n')
    
    file_size = os.path.getsize(filepath) / 1024 / 1024
    saved_files.append((year, len(yearly_data[year]), file_size))
    print(f"✅ {filename}: {len(yearly_data[year])}개 문서, {file_size:.2f} MB")

print(f"\n총 {len(saved_files)}개 파일 저장 완료!")

✅ documents_2010.cmds: 841개 문서, 39.75 MB
✅ documents_2011.cmds: 855개 문서, 51.12 MB
✅ documents_2012.cmds: 1280개 문서, 70.65 MB
✅ documents_2013.cmds: 1582개 문서, 77.97 MB
✅ documents_2014.cmds: 1504개 문서, 70.57 MB
✅ documents_2015.cmds: 1367개 문서, 49.90 MB
✅ documents_2016.cmds: 1528개 문서, 48.76 MB
✅ documents_2017.cmds: 1963개 문서, 64.14 MB
✅ documents_2018.cmds: 1661개 문서, 56.05 MB
✅ documents_2019.cmds: 1478개 문서, 53.26 MB
✅ documents_2020.cmds: 1075개 문서, 27.03 MB
✅ documents_2021.cmds: 1331개 문서, 43.26 MB
✅ documents_2022.cmds: 1818개 문서, 69.17 MB
✅ documents_2023.cmds: 1442개 문서, 49.07 MB
✅ documents_2024.cmds: 1811개 문서, 60.20 MB
✅ documents_2025.cmds: 1784개 문서, 62.33 MB

총 16개 파일 저장 완료!


In [10]:
# 파싱 에러 문서 별도 저장 (있는 경우)
if parse_errors:
    error_filepath = os.path.join(OUTPUT_DIR, 'documents_parse_error.cmds')
    with open(error_filepath, 'w', encoding='utf-8') as f:
        for line_no, err, preview in parse_errors:
            # 원본 줄 저장
            orig_line = lines[line_no - 1].rstrip('\n')
            f.write(orig_line + '\n')
    print(f"⚠️ 파싱 에러 문서 저장: {error_filepath}")
else:
    print("파싱 에러 없음")

파싱 에러 없음


## 완료!

### 결과
- `yearly/documents_YYYY.cmds` - 연도별 파일 (sourceId 오름차순 정렬)
- `yearly/documents_parse_error.cmds` - 파싱 에러 문서 (있는 경우)